colors: Number of colors

C: number of classes

A: actions {l:left, r:right, a:above, b:below}

cube: adjacency matrix of the cube


In [1]:
import tensorflow as tf
import numpy as np

Build the cube adjancy matrix


In [14]:
cube = np.array([[1, 3, 4, 5], [2,0,4,5], [3,1,4,5], [0,2,4,5], [1,3,2,0], [3,1,2,0]])

* generate a dataset
* Assume that we are generating 10 classes, with 10 colors in total

In [3]:
batch_size = 10
gamma = 0.9
num_colors = 10
num_classes = 10
num_faces = 6
SMALL_NUM = 1e-10
initLr = 5e-3
lrDecayRate = .995
lrDecayFreq = 500
momentumValue = .9

In [4]:
X = np.random.randint(batch_size, size=(batch_size, num_faces))
y = np.arange(batch_size)
# converting in one-hot representation
X_train = np.zeros((batch_size, num_faces, num_colors))
for i in range(batch_size):
    for j in range(num_faces):
        X_train[i, j, X[i, j]] = 1

# Make the y one-hot representation
y_train = np.zeros((batch_size, num_classes))
for i in range(batch_size):
    y_train[i, y[i]] = 1

* Define the computation graph

In [5]:
def get_kernel(kernel_shape, bias_shape):
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer())
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
                             initializer=tf.random_normal_initializer())
    return weights, biases

In [13]:
def get_next_batch(x, init_view, actions, cube):
    """ Get the slice of the next input from the placeholder x
    given the next location (left, right, above, below) to look at.
    One thing to make sure that you need to first sample to face index
    from the cube adjacency matrix, given the current face and the action
    taken. 
    : param x: complete data matrix, of shape: (batch_size, num_faces, num_colors)
    : param init_view: the current faces in the batch_x
    : param actions: the action taken (l, r, a, b)
    : param cube: the cube data adjacency matrix.
    : return batch_x: the next_batch 
    """
    # from actions, find the face index
#     face_index = []
    size = [1, num_colors]
    batch_x = []
    for i in range(batch_size):
        face_index = cube[init_view[i, 0], actions[i, 0]]
        begin = [face_index, 0]
        z = tf.slice(x[i, :, :], begin=begin, size=size)
        z = tf.reshape(z, shape=[tf.shape(z)[-1]])
        batch_x.append(z)
    batch_x = tf.pack(next_batch, axis=0)
    return batch_x

def get_init_view(x, init_view):
    """Given the complete data matrix, give me the batch_x
    :param x: complete data matrix, of shape: (batch_size, num_faces, num_colors)
    :param init_view: the indices of the faces of the cube that 
    you see on the first time
    :return batch_x: the batch for learning of shape (batch_size x num_colors)
    the colors are represented in one-hot way.
    """
    init_view = tf.argmax(init_view, )
    size = [1, num_colors]
    batch_x = []
    for i in range(batch_size):
        begin = [init_view[i, 0], 0]
        z = tf.slice(x[i, :, :], begin=begin, size=size)
        z = tf.reshape(z, shape=[tf.shape(z)[-1]])
        batch_x.append(z)
    batch_x = tf.pack(next_batch, axis=0)
    return batch_x

In [7]:
def model():
    x = tf.placeholder(name="input_data", shape=[None, num_faces, num_colors], dtype=tf.float32) # total number of colors are 10 in one hot way
    y = tf.placeholder(name="labels", shape=[None, num_classes], dtype=tf.float32)
    cube = tf.constant(cube)
    COSTS = []
    Rewards = []
    outputs = []
    REUSE = False
    samp_actions = []
    ys = []

    # randomly generate first input view location.
    init_view = tf.random_uniform((batch_size, 1), minval=0, maxval=5.99)
    init_view = tf.to_int32(init_view, name='ToInt32')
    batch_x = get_init_view(x, init_view)
    for i in range(num_faces):
        with tf.variable_scope("hidden", reuse=REUSE):
            W, b = get_kernel(kernel_shape=[10, 30], bias_shape=[30])

        with tf.variable_scope("class", reuse=REUSE):
            W_c, b_c = get_kernel(kernel_shape=[30, 10], bias_shape=[10])

        with tf.variable_scope("action", reuse=REUSE):
            W_a, b_a = get_kernel(kernel_shape=[30, 4], bias_shape=[4])

            
        with tf.name_scope("hidden_proj") as scope:
            # Construct a linear model
            hidden = tf.matmul(batch_x, W) + b
        
        with tf.name_scope("classification") as scope:
            probs = tf.nn.softmax(tf.matmul(hidden, W_c) + b_c)  # Softmax prob for classes
        
        mean_actions = tf.nn.softmax(tf.matmul(hidden, W_a) + b_a)  # Softmax prob for actions
        tf.stop_gradient(mean_actions)
        
        # sample from these actions
        samp_action = tf.multinomial(mean_actions, 1,)
        samp_action = tf.cast(samp_action, dtype=tf.int32)
        tf.stop_gradient(samp_action)
        
        # define the rewards
        max_p_y = tf.arg_max(probs, 1)
        max_y = tf.arg_max(y, 1)

        equals = tf.equal(max_p_y,max_y)
        equals = tf.cast(equals, tf.float32)
        acc = tf.reduce_mean(equals)
        
        R = equals
        R = tf.reshape(R, (batch_size, 1))
        
        # Loss function corresponding to the policy
        # I guess this is the probility of the action getting selected under softmax assumption
        with tf.name_scope("policy_gradient") as scope:
#             p_loc = tf.zeros_like(samp_action, dtype=tf.float32)
            p_loc = []
            for index in range(batch_size):
                p_loc.append(mean_actions[i, samp_action[i, 0]])
            p_loc = tf.pack(p_loc, axis=0)
            loss_p = tf.log(p_loc) * (R) #*gamma**i)
        # Loss function correponding to the class  lychee fruit.prediction
        with tf.name_scope("class_loss") as scope:
            # Minimize error using cross entropy, y is in one-hot representation
            loss_class = y * tf.log(probs)
            
        J = tf.concat(1, [loss_p, loss_class])
        J = tf.reduce_sum(J, 1)
        J = tf.reduce_mean(J, 0)

        cost = -J
        COSTS.append(cost)
        samp_actions.append(samp_action)
        accuracies.append(acc)
        Rewards.append(R)
#         outputs.append()
        
        # generate new views
        batch_x = get_next_batch(x, batch_x, samp_action, cube)
        
        # Don't use the initialized variable again
        REUSE = True
        
    COSTS = tf.reduce_sum(COSTS)
    reward = tf.reduce_mean(Rewards)
    accuracy = tf.reduce_sum(accuracies)
    with tf.name_scope("train") as scope:
        # Gradient descent
        optimizer = tf.train.GradientDescentOptimizer(lr).minimize(COSTS)
    return x, y, COSTS, reward, accuracy

In [8]:
accuracies = []
global_step = tf.Variable(0, trainable=False)
lr = tf.train.exponential_decay(initLr, global_step, lrDecayFreq, lrDecayRate, staircase=True)
x, y, costs, reward, acc = model()

In [ ]:
# with tf.Session() as sess:
#     v = tf.constant(value=np.zeros((10, 6, 10)))
# #     zero = tf.constant(np.arange(2))
# #     first = tf.constant(np.arange(10))
# #     last = tf.constant(np.arange(6))
# #     z = v[first, zero, last]
# #     indices = tf.constant(5)
# # #     zero = tf.constant(0)
# #     begin = tf.pack([indices, 0])
# #     shape_ = tf.constant([1, 10])
# #     zs = []
# #     for i in range(10):
# #         z = tf.slice(v[i, 1, :], begin=[indices, 0], size=[1, 10])
# #         shape_z = tf.shape(z)
# #         z = tf.reshape(z, shape=[shape_z[-1]])
# #         zs.append(z)
# #     tf.global_variables_initializer()
# #     zs = tf.pack(zs, axis=0)
#     l = tf.constant([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])
#     zs = v[0, :, l]
#     print (sess.run(zs).shape)